"""

This notebook runs through Section 2.3: Style Transfer. 
The images generated are similar to those in Figure 3. 

The Directory Paths block should be edited to suit local directory structure. 
The Chosen Parameters block can be changed to try out different experimental settings. 

"""

In [ ]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import tensorflow as tf
import tensorflow.contrib.slim as slim

from utils.imaging import display_image, format_image, save_image
from utils.losses import gram_matrix, style_layer_loss
import vgg

### Directory Paths

In [ ]:
# Path to images we are extracting content and style from
content_image_path = './images/perth_skyline.jpg'
style_image_path = './images/starry_night.jpg'

# Path to vgg19 checkpoint, must be downloaded separately
checkpoint_path = './vgg_19.ckpt'

# Location of tensorboard summaries
tensorboard_dir = './train/'

# Path to directory used for storing images
debug_dir = './debug/'

# Determines whether information is saved between runs
# for tensorboard
reset_saves = True
if reset_saves is True:
    # Ensure tensorboard is not running when you try to delete
    # this directory
    if os.path.exists(tensorboard_dir):
        shutil.rmtree(tensorboard_dir)
        
# Create the debug directory if it doesn't exist
# Tensorboard directory is made automatically if it doesn't exist
if os.path.exists(debug_dir):
    shutil.rmtree(debug_dir)
os.makedirs(debug_dir)

### Chosen Parameters

In [ ]:
# Dimensions desired for image, channels must be kept as 3
height = 224
width = 224
channels = 3

# Set the seeds to provide consistency between runs
# Can also comment out for variability between runs
np.random.seed(0)
tf.set_random_seed(0)

# Layer being for content component
content_layer = 'vgg_19/conv4/conv4_2'

style_list = ['vgg_19/conv1/conv1_1', 'vgg_19/conv2/conv2_1', 'vgg_19/conv3/conv3_1',
    'vgg_19/conv4/conv4_1', 'vgg_19/conv5/conv5_1']

# Chosen depth corresponds to how many feature layers you want to use
# for the style component
chosen_depth = 5

# Weights for each loss component
content_weight = 1.0
style_weight = content_weight/8e-4

# Learning rate for optimizer
learning_rate = 1e-1

# Number of training and validation step
# In this instance, validation refers to when we would like to examine:
# currently optimized image and loss
training_steps = 5000000
validation_steps = 100000

# Online debugging refers to images that will be displayed within the notebook 
# using plt, every validation step
# Offline debugging refers to images that will be saved to folder using plt, 
# every validation step
debug_online = True
debug_offline = True

### Set up the input node and feature extractor

In [ ]:
# The input node to the graph
# These values are what is required by vgg19 for height, width, channels
input_var_initial_value = np.random.rand(1, height, width, channels)
input_var = tf.Variable(input_var_initial_value, dtype=tf.float32, name='input_var')

# Load the vgg model
with slim.arg_scope(vgg.vgg_arg_scope()):
    end_points = vgg.vgg_19_conv(input_var)

### Set up saver for vgg_19 variables only

In [ ]:
# Prepare to restore the vgg19 nodes
# Skip trying to restore the input variable since it's new
all_variables = tf.get_collection_ref(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list=all_variables[1:])

### Load real images and create to-be-optimized image

In [ ]:
# Construct the content image tensor
# And the graph operation which assigns it to input_var
content_image = plt.imread(content_image_path)
content_image = cv2.resize(content_image, (height, width))
content_image_batch = np.expand_dims(content_image, axis=0)
content_image_batch = np.asarray(content_image_batch, dtype=np.float32)
content_image_tensor = tf.Variable(content_image_batch, dtype=tf.float32, name='content_image')

assign_content_image = tf.assign(input_var, content_image_tensor, name='assign_content_image')

# Construct the style image tensor
# And the graph operation which assigns it to input_var
style_image = plt.imread(style_image_path)
style_image = cv2.resize(style_image, (height, width))
style_image_batch = np.expand_dims(style_image, axis=0)
style_image_batch = np.asarray(style_image_batch, dtype=np.float32)
style_image_tensor = tf.Variable(style_image_batch, dtype=tf.float32, name='style_image')

assign_style_image = tf.assign(input_var, style_image_tensor, name='assign_style_image')

# Construct the white noise tensor
# And the graph operation which assigns it to input_var
white_noise = np.random.rand(height, width, channels) * 255.
white_noise_batch = np.expand_dims(white_noise, axis=0)
white_noise_batch = np.asarray(white_noise_batch, dtype=np.float32)
white_noise_tensor = tf.Variable(white_noise_batch, dtype=tf.float32, name='white_noise')

assign_white_noise = tf.assign(input_var, white_noise_tensor, name='assign_white_noise')

### Set up content extraction node

In [ ]:
content_rep = end_points[content_layer]

### Obtain content representation for real image

In [ ]:
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(),
                  name='initialize_all')

with tf.Session() as sess:
    # Initialize new variables and then restore vgg_19 variables
    sess.run(init_op)
    saver.restore(sess, checkpoint_path)
    
    assign_content_image.eval()
    
    real_content_rep_ = sess.run(content_rep)
    
real_content_rep = tf.constant(real_content_rep_, name='real_content_rep')

### Calculate the content loss

In [ ]:
content_loss = tf.losses.mean_squared_error(labels=real_content_rep, predictions=content_rep)

### Set up gram matrix nodes

In [ ]:
grams = []
filter_sizes = []
for i in range(chosen_depth):
    chosen_layer = end_points[style_list[i]]
    gram_features = gram_matrix(chosen_layer)
    grams.append(gram_features)
    
    # Determine the size of the filters used at each layer
    # This is needed to calculate the loss from that layer
    _, filter_height, filter_width, _ = chosen_layer.get_shape().as_list()
    filter_size = float(filter_height * filter_width)
    filter_sizes.append(filter_size)    

### Obtain gram matrices for real image

In [ ]:
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(),
                  name='initialize_all')

with tf.Session() as sess:
    # Initialize new variables and then restore vgg_19 variables
    sess.run(init_op)
    saver.restore(sess, checkpoint_path)
    
    assign_style_image.eval()
    
    real_image_grams = sess.run(grams)
    
# Create constants with the real image gram matrices
gram_constants = []
for i in range(chosen_depth):
    node_name = 'gram_constant_{}'.format(i)
    gram_constant = tf.constant(real_image_grams[i], name=node_name)
    gram_constants.append(gram_constant)

### Calculate the style loss

In [ ]:
layer_losses = []
for i in range(chosen_depth):
    layer_loss = style_layer_loss(gram_constants[i], grams[i], filter_sizes[i])
    # Equal weighting on each loss, summing to 1
    layer_loss *= (1.0 / chosen_depth)
    layer_losses.append(layer_loss)
style_loss = tf.add_n(layer_losses, name='sum_layer_losses')

### Set up the final loss, optimizer, and summaries

In [ ]:
loss = (content_weight * content_loss) + (style_weight * style_loss)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss, var_list=[input_var])

# Initializers
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(),
                  name='initialize_all')

# Tensorboard summaries
loss_summary = tf.summary.scalar('loss', loss)
image_summary = tf.summary.image('image', input_var)

### Training

In [ ]:
with tf.Session() as sess: 
    # Initialize all variables and then
    # restore weights for feature extractor
    sess.run(init_op)
    saver.restore(sess, checkpoint_path)
    
    # Set up summary writer for tensorboard, saving graph as well
    train_writer = tf.summary.FileWriter(tensorboard_dir, sess.graph)
    
    # Assign the white noise to the variable being optimized
    assign_white_noise.eval()
    
    # Begin training
    for i in range(training_steps):
        loss_summary_, _ = sess.run([loss_summary, train_op])
        train_writer.add_summary(loss_summary_, i)
        
        if i % validation_steps == 0:
            image_summary_, current_image, loss_ = sess.run(
                [image_summary, input_var, loss])
            
            train_writer.add_summary(image_summary_, i)
            
            print('Step: {}, Loss: {}'.format(i, loss_))
            
            if debug_online is True:
                display_image(current_image, i)
            if debug_offline is True:
                save_image(current_image, i, debug_dir)  
                
    # Display and save the final image
    current_image = input_var.eval()
    display_image(current_image, 'Final')
    save_image(current_image, 'Final', debug_dir)